In [34]:
import pandas as pd
import datetime
from minio import Minio

import sys
sys.path.append(r'D:\Documentos\Projetos\Ollist_End_to_End\airflow\dags')
import functions as f

In [35]:
data_lake_server= f.var['data_lake_server_local']
data_lake_login= f.var['data_lake_login']
data_lake_password= f.var['data_lake_password']

client = Minio(
        endpoint= data_lake_server,
        access_key= data_lake_login,
        secret_key= data_lake_password,
        secure=False
    )

# Importando dados

## Clientes

In [36]:
customers = pd.read_parquet(f.read_minio_file(bucket_name= 'processing', file= 'olist_customers_dataset.parquet', client= client))
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## Pedidos

In [37]:
orders = pd.read_parquet(
    f.read_minio_file(bucket_name= 'processing', file= 'olist_orders_dataset.parquet', client= client)
)

orders.order_purchase_timestamp = pd.to_datetime(orders.order_purchase_timestamp)
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


## Itens de Pedidos

In [38]:
order_items = pd.read_parquet(
    f.read_minio_file(bucket_name= 'processing', file= 'olist_order_items_dataset.parquet', client= client)
)

order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


## Geolocalização

In [39]:
geo = pd.read_csv(
    f.read_minio_file(bucket_name= 'landing', file= 'olist_geolocation_dataset.csv', client= client)
)

geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


# Validando dados

## customers

In [40]:
customers.sample()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
53966,47c47d5f6c02ce30ae9d9916399982dc,91b4967f242b083dd37c8e21469898ed,1210,sao paulo,SP


In [41]:
print('Há {} linhas e {} clientes únicos'.format(customers.shape[0], len(customers.customer_id.unique())))

Há 99441 linhas e 99441 clientes únicos


## orders

In [42]:
orders.sample()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
21454,844ec50f4de4b51c2905edc92cfff82d,c231d1c8dfe0c0c263f5276e21a21545,delivered,2017-11-24 08:39:30,2017-11-25 04:30:55,2017-11-27 20:36:32,2017-11-28 23:41:34,2017-12-08 00:00:00


In [43]:
print('Há {} pedidos'.format(orders.order_id.unique().shape[0]))

Há 99441 pedidos


In [44]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

 - Não há valores nulos nas colunas de **'customer_id'** e **'order_purchase_timestamp'**

In [45]:
print('Há {} clientes únicos na base de pedidos'.format(orders.customer_id.unique().shape[0]))

Há 99441 clientes únicos na base de pedidos


- O número é igual a quantidade de clientes na base de clientes

#### Última compra registrada

In [46]:
ult_compra = orders.order_purchase_timestamp.max()

## order_items

In [47]:
order_items.sample()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
23540,35ae6fcf66dba4f6760a20b921867b2b,2,e5ae72c62ebfa708624f5029d609b160,88460e8ebdecbfecb5f9601833981930,2018-07-31 15:04:14,59.9,1.98


In [48]:
order_items.shape

(112650, 7)

In [49]:
order_items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

 - Não há valores nulos

In [50]:
print('Há {} pedidos únicos na base de itens de pedido'.format(order_items.order_id.unique().shape[0]))

Há 98666 pedidos únicos na base de itens de pedido


#### Porque há menos pedidos na base de itens que na base de pedidos?

In [51]:
list_order_items = order_items.order_id.to_list()

In [52]:
orders[~orders.order_id.isin(list_order_items)].order_status.value_counts()

unavailable    603
canceled       164
created          5
invoiced         2
shipped          1
Name: order_status, dtype: int64

- Dessa forma podemos validar que os pedidos sem preço se tratam de pedidos não entregues ainda. Assim, serão descartados na análise de agrupamento de clientes

## geo

In [53]:
geo[['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']].duplicated().sum()

972251

# Agrupamento de Clientes

## Criando bases auxiliares

### Preço total de compras

In [54]:
price_per_order = order_items.groupby('order_id').price.sum().reset_index().rename(columns= {'price': 'monetary'})

orders = pd.merge(orders, price_per_order, on= 'order_id', how= 'inner')

### Data da última compra

In [55]:
orders['days_ult_compra'] = (ult_compra - orders.order_purchase_timestamp).dt.days

### Unindo customers e geolocation

In [56]:
geo_means = geo.groupby(
        ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']
    )[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

customers = customers.merge(
    geo_means,
    left_on= ['customer_zip_code_prefix', 'customer_city', 'customer_state'],
    right_on= ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'],
    how= 'left').drop(columns= ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'])

del geo_means

## Dataframe RFM

In [65]:
df_rfm = \
pd.merge(
    customers[['customer_unique_id', 'customer_id', 'geolocation_lat', 'geolocation_lng']],
    orders[['customer_id', 'monetary', 'days_ult_compra']],
    on= 'customer_id',
    how= 'left')\
.groupby('customer_unique_id')\
.agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean',
    'customer_id': 'count',
    'monetary': 'sum',
    'days_ult_compra': 'min'
}).reset_index()\
.rename(
    columns= {
        'customer_id': 'frequency',
        'days_ult_compra': 'recency'
    }
)

#### Validação dos valores

In [66]:
df_rfm.sample()

,customer_unique_id,geolocation_lat,geolocation_lng,frequency,monetary,recency
55419,93bc212addb25a5f5139fded3c2ee6b3,-28.847935,-51.891517,1,1519.46,377.0


In [67]:
id_order = 'b62777d3969196b3329205b4d8e1ba0a'

In [68]:
customer_id_list = customers[customers.customer_unique_id == id_order].customer_id.to_list()
orders[orders.customer_id.isin(customer_id_list)]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,monetary,days_ult_compra
10830,bfcf79aaae774aa4030a0d2e2046c525,a27429e671f53ed883053fd2c9e33f1f,delivered,2017-08-10 13:44:51,2017-08-10 14:04:02,2017-08-17 16:52:37,2017-08-23 18:03:16,2017-09-05 00:00:00,31.99,433
54652,b3bb81a4fe1e47c62f92706ebdbf4e4b,b6f70fd2cf64a0aaa63332651fc24d3b,delivered,2017-08-07 19:07:37,2017-08-07 19:23:36,2017-08-10 16:40:38,2017-08-21 22:11:03,2017-08-31 00:00:00,22.99,435


In [69]:
order_list = orders[orders.customer_id.isin(customer_id_list)].order_id.to_list()
order_items[order_items.order_id.isin(order_list)]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
79018,b3bb81a4fe1e47c62f92706ebdbf4e4b,1,b0c1e71a37e29bd3d463eb93e4861c6a,1caf283236cd69af44cbc09a0a1e7d32,2017-08-11 19:23:36,22.99,16.11
84425,bfcf79aaae774aa4030a0d2e2046c525,1,c982dbea53b864f4d27c1d36f14b6053,1caf283236cd69af44cbc09a0a1e7d32,2017-08-16 14:04:02,31.99,16.11
